In [1]:
!pip install vaderSentiment
!pip install tensorflow-datasets

# scikilearn.__version__ == 0.22.2

In [2]:
import pickle
import joblib
import string
import pandas as pd
import numpy as np
from string import punctuation
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#nltk
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from nltk.tokenize import word_tokenize,sent_tokenize
import numpy as np
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

from sklearn.svm import SVC
import tensorflow as tf
from keras.datasets import imdb

[nltk_data] Downloading package punkt to /Users/ccy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/ccy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ccy/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/ccy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# 1. Benchmark - Using VadarSentiment

In [3]:
# New words and values to update the Lexicon.
new_words = {
    'crushes': 10,
    'beats': 5,
    'increase':10,
    'increasing':10,
    'long': 50,
    'misses': -5,
    'trouble': -10,
    'falls': -100,
    'drops':-100,
    'dropping':-200,
    'falling':-100
}

In [4]:
analyser = SentimentIntensityAnalyzer()
# Example
score = analyser.polarity_scores("Apple Stock Is Falling Again. Why That’s Not a Problem for the Dow.")
print(score)
analyser.lexicon.update(new_words)
# Example
score = analyser.polarity_scores("Apple Stock Is Falling Again. Why That’s Not a Problem for the Dow.")
print(score)

{'neg': 0.108, 'neu': 0.74, 'pos': 0.152, 'compound': 0.1675}
{'neg': 0.884, 'neu': 0.096, 'pos': 0.02, 'compound': -0.9992}


# 2. Using LSTM model trained on IMDB dataset from tensorflow

In [5]:
def pad_to_size(vec, size):
    zeros = [0] * (size - len(vec))
    vec.extend(zeros)
    return vec

def sample_predict(sample_pred_text, encoder, pad, model):
    encoded_sample_pred_text = encoder.encode(sample_pred_text)

    if pad:
        encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, 64)
    encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
    predictions = model.predict(tf.expand_dims(encoded_sample_pred_text, 0))

    return (predictions)



## Load model

In [6]:
encoder_loaded = pickle.load(open("saved_model/lstm_encoder.pickel", "rb"))
lstm_loaded = tf.keras.models.load_model('saved_model/lstm')

# Check its architecture
lstm_loaded.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          523840    
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         66048     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 635,329
Trainable params: 635,329
Non-trainable params: 0
__________________________________________________

In [7]:
new_prediction = sample_predict("Apple Stock Is Falling Again. Why That’s Not a Problem for the Dow.", encoder = encoder_loaded, pad=False, model = lstm_loaded)
print(new_prediction)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[[-0.5807369]]


# 3. Building a Neural Network and train on IMDB Dataset

## Text preprocessing

In [8]:
TOP_WORDS = 10000

In [9]:
stop_words = set(stopwords.words("english"))
ps = PorterStemmer()

In [10]:

word2index = imdb.get_word_index()
word2index = {k:(v+3) for k,v in word2index.items()}
#word_to_id["<PAD>"] = 0
#word_to_id["<START>"] = 1
#word_to_id["<UNK>"] = 2
# Tweet preprocessing
def clean_symbols(text):
    
    for char in text:
        # remove punctuation but preserve symbols defined above 
        if char in string.punctuation and char != ' ':
            text = text.replace(char, '')
        # remove all other characters
        if char.isalpha() is False and char.isdigit() is False and char != ' ':
            text = text.replace(char, '')
          
    return ' '.join(text.split())

def clean_words(news, dimension = TOP_WORDS):
    cleaned = clean_symbols(news).lower()
    test=[]
    for word in word_tokenize(cleaned):
        if word in word2index:
            test.append(word2index[word])
    
    results = np.zeros(dimension)
    for _ , sequence in enumerate(test):
        if sequence < dimension:
            results[sequence] = 1
    
    #print("\nOriginal string:", news,"\n")
    #print("\nIndex conversion:", test,"\n")
    results = np.reshape(results,(1, TOP_WORDS))
    #print("\nConvert to vectors:", results,"\n")
    return results


      

## Example

In [11]:
example_1 = "The movie was not good. The animation and the graphics were terrible. I would not recommend this movie."
example_2 = "it is so good"
example_3 = "Apple Stock Is Falling Again. Why That’s Not a Problem for the Dow."
def predict_sentiment(x_test, NN):

    x_test = clean_words(x_test, dimension = TOP_WORDS)
    #print(x_test.shape)
    prediction = NN.predict(x_test) 
    #print(prediction)
    return prediction

## Load the model

In [12]:
NN_loaded = tf.keras.models.load_model('saved_model/nn')

# Check its architecture
NN_loaded.summary()

predict_sentiment("it is good and let's have a try", NN_loaded)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 50)                500050    
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_5 (Dense)              (None, 16)                816       
_________________________________________________________________
dropout_3 (Dropout)          (None, 16)               

array([[0.98693943]], dtype=float32)

# SVC trained on another dataset

In [13]:
# making list stopwords for removing stopwords from our text 
stop = set(stopwords.words('english'))
stop.update(punctuation)
print(stop)

{'his', 'in', "it's", 'into', 'yourselves', 'any', "couldn't", 'because', 'will', 'here', "she's", 'we', '\\', 'just', 'herself', 'o', 'after', 'and', 'whom', '@', 'me', 'what', "weren't", 'him', 'which', 'on', 'again', 'while', 'was', 'their', "wouldn't", "'", ';', 'same', 'hasn', 'or', 'weren', 'an', 'ma', 'is', 'them', 'it', 'out', 'himself', '$', 'can', 'her', 'with', 'further', "mustn't", '&', 'i', ',', ':', '<', 'you', 'of', 'these', 'myself', 'should', 'how', 'needn', 'why', "shan't", 'has', 'd', 'ourselves', "haven't", 'be', '[', 'had', 'own', 'at', 'more', 'against', 'through', 'until', 'below', "needn't", 'where', 'some', '_', "should've", '*', 'if', 'so', 'll', "wasn't", 'very', 'not', 'as', 'up', 'wasn', "won't", 'both', '"', 'those', 'hers', 'being', '#', 'such', 'that', 've', 'mightn', 'this', 'he', '`', 'ain', 'there', '.', 'don', 'down', '%', 'shan', "doesn't", 'does', 'now', "that'll", 'couldn', 'doing', 'they', 'having', 'between', ')', 'a', 'before', 'above', "should

In [14]:
# this function return the part of speech of a word.
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [15]:
# Function to clean our text.
lemmatizer = WordNetLemmatizer()
def clean_review(text):
    clean_text = []
    for w in word_tokenize(text):
        if w.lower() not in stop:
            pos = pos_tag([w])
            new_w = lemmatizer.lemmatize(w, pos=get_simple_pos(pos[0][1]))
            clean_text.append(new_w)
    return clean_text

def join_text(text):
    return " ".join(text)

## Load the model

In [16]:
# load pickle
count_vec_loaded = pickle.load(open("saved_model/count_vector.pickel", "rb"))

# load
svc_loaded = joblib.load("saved_model/svc.pkl")

In [17]:
def svc_predict(svc, count_vec, news):
    news_processed = clean_review(news)
    news_processed = join_text(news_processed)
    
    test_news = count_vec.transform([news_processed]).todense()
    return svc.predict_proba(test_news)

In [18]:
print(svc_predict(svc_loaded, count_vec_loaded, "Investors are looking to buy more stocks"))

[[0.02619572 0.69802667 0.27577761]]


# Combine all models together to give the final rating

In [95]:
def predict_rating(news, if_print = False):
    benchmark_rating = analyser.polarity_scores(news)['compound']

    # Predicted by LSTM model, using padding = True
    rating_lstm = sample_predict(news, encoder_loaded, pad=True, model = lstm_loaded)[0][0]
  
    # predicted by NN
    rating_NN = predict_sentiment(news, NN_loaded)[0][0]
    
    # predictd by SVC
    rating_svc = svc_predict(svc_loaded, count_vec_loaded, news)[0]
    
    if if_print:
        print(f'result from vadar: {benchmark_rating:.2f}')
        print(f'result from lstm: {rating_lstm:.2f}')
        print(f'result from Neural Network: {rating_NN:.2f}')
        print(f"result from SVC: positive:{rating_svc[2]:.2f}, neutral:{rating_svc[1]:.2f}, negative:{rating_svc[0]:.2f}")
        
    if (benchmark_rating < 0.5 and rating_lstm <0 and rating_NN < 0.5) or benchmark_rating < 0 or rating_NN < 0.2 or (rating_svc[0]>0.5 and np.argmax(rating_svc) == 0):
        return "Strong Negative"
    elif (benchmark_rating >= 0.5 and rating_lstm >=0 and rating_NN >= 0.5) or (benchmark_rating >= 0.8 or rating_NN >= 0.8) or (rating_svc[2] > 0.5 and np.argmax(rating_svc) == 2):
        return "Strong Positive"
    elif benchmark_rating >= 0.5 and rating_lstm >=0 and rating_svc[-1] > rating_svc[0]:
        return "Positive"
    elif benchmark_rating <0.5 and rating_lstm <0 or rating_svc[0] > rating_svc[-1]:
        return "Negative"
    else:
        return "Neutral"

In [96]:
predict_rating("Apple Stock Is Falling Again. Why That’s Not a Problem for the Dow.", True)

result from vadar: -1.00
result from lstm: 1.11
result from Neural Network: 0.00
result from SVC: positive:0.09, neutral:0.85, negative:0.06


'Strong Negative'

In [97]:
predict_rating("Apple Stock displays good prospect in the future, investors highs high")

'Strong Positive'

In [98]:
predict_rating("Apple New iphone is coming")


'Strong Positive'

In [91]:
predict_rating("The market is falling")

'Strong Negative'

## Stock news

In [118]:
## referene: https://towardsdatascience.com/stock-news-sentiment-analysis-with-python-193d4b4378d4

In [166]:
# Import libraries
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from urllib.request import urlopen, Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Parameters 
n = 10 #the # of article headlines displayed per ticker
tickers = ['AAPL', 'TSLA']

# Get Data
finwiz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}

for ticker in tickers:
    url = finwiz_url + ticker
    req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
    resp = urlopen(req)    
    html = BeautifulSoup(resp, features="lxml")
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table

try:
    for ticker in tickers:
        df = news_tables[ticker]
        df_tr = df.findAll('tr')
    
        print ('\n')
        print ('Recent News Headlines for {}: '.format(ticker))
        
        for i, table_row in enumerate(df_tr):
            a_text = table_row.a.text
            td_text = table_row.td.text
            td_text = td_text.strip()
            print(a_text,'(',td_text,')')
            if i == n-1:
                break
except KeyError:
    pass

# Iterate through the news
parsed_news = []
for file_name, news_table in news_tables.items():
    for x in news_table.findAll('tr'):
        print(x)
        source = x.span.get_text()
        text = x.a.get_text() 
        date_scrape = x.td.text.split()
        news_url = x.a['href']
        
        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        else:
            date = date_scrape[0]
            time = date_scrape[1]
    
            
        ticker = file_name.split('_')[0]
        
        parsed_news.append([ticker, date, time, text, news_url, source])
    
# Sentiment Analysis
analyzer = SentimentIntensityAnalyzer()

columns = ['Ticker', 'Date', 'Time', 'Headline', 'Link', 'Source']
news = pd.DataFrame(parsed_news, columns=columns)
scores = news['Headline'].apply(analyzer.polarity_scores).tolist()

df_scores = pd.DataFrame(scores)
news = news.join(df_scores, rsuffix='_right')

sentiments = news['Headline'].apply(predict_rating).tolist()

df_sentiments = pd.DataFrame(sentiments)
news = news.join(df_sentiments, rsuffix='_right')

# View Data 
news['Date'] = pd.to_datetime(news.Date).dt.date

unique_ticker = news['Ticker'].unique().tolist()
news_dict = {name: news.loc[news['Ticker'] == name] for name in unique_ticker}

values = []
for ticker in tickers: 
    dataframe = news_dict[ticker]
    dataframe = dataframe.set_index('Ticker')
    print ('\n')
    print (dataframe.head())
    mean = round(dataframe['compound'].mean(), 2)
    values.append(mean)
    
df = pd.DataFrame(list(zip(tickers, values)), columns =['Ticker', 'Mean Sentiment']) 
df = df.set_index('Ticker')
df = df.sort_values('Mean Sentiment', ascending=False)
print ('\n')
print (df)





Recent News Headlines for AAPL: 
One-Time Pioneer LG Electronics to Close Smartphone Business ( Apr-04-21 10:20PM )
Why Netflix $450M 'Knives Out' gambit is becoming routine as streaming platforms 'feed the beast' ( 02:04PM )
3 Bullish Tech Stocks For Technical Traders Going Into The Week ( 01:57PM )
3 Dividend-Paying Tech Stocks to Buy Right Now ( 08:36AM )
Benzinga's Bulls And Bears Of The Week: Apple, GM, JetBlue, Lululemon, Tesla And More ( Apr-03-21 11:20AM )
Fortnite maker Epic kicks off UK lawsuit against Google ( Apr-02-21 07:07PM )
Tech Has a Starring Role in Bidens Infrastructure Plan. Here Are 15 Stocks That Could Benefit. ( 06:41PM )
You May Not Like Facebook, but Its Stock Deserves Betterat Least 20% Better ( 05:38PM )
7 Robinhood Stocks Even Billionaires Want in Their Portfolios ( 11:15AM )
New Investor? Here Are 3 Great Starter Stocks ( 10:00AM )


Recent News Headlines for TSLA: 
Dow Jones Futures Rise As Tesla, Jobs Report Bolster Stock Market Rally; Google, Microsof

In [167]:
news

,Ticker,Date,Time,Headline,Link,Source,neg,neu,pos,compound,0
0,AAPL,2021-04-04,10:20PM,One-Time Pioneer LG Electronics to Close Smart...,https://finance.yahoo.com/news/one-time-pionee...,Bloomberg,0.000,1.000,0.000,0.0000,Strong Negative
1,AAPL,2021-04-04,02:04PM,Why Netflix $450M 'Knives Out' gambit is becom...,https://finance.yahoo.com/news/why-netflix-450...,Yahoo Finance,0.000,1.000,0.000,0.0000,Strong Negative
2,AAPL,2021-04-04,01:57PM,3 Bullish Tech Stocks For Technical Traders Go...,https://finance.yahoo.com/news/3-bullish-tech-...,Benzinga,0.000,1.000,0.000,0.0000,Strong Negative
3,AAPL,2021-04-04,08:36AM,3 Dividend-Paying Tech Stocks to Buy Right Now,https://www.fool.com/investing/2021/04/04/3-di...,Motley Fool,0.000,1.000,0.000,0.0000,Neutral
4,AAPL,2021-04-03,11:20AM,"Benzinga's Bulls And Bears Of The Week: Apple,...",https://finance.yahoo.com/news/benzingas-bulls...,Benzinga,0.000,1.000,0.000,0.0000,Strong Positive
...,...,...,...,...,...,...,...,...,...,...,...
195,TSLA,2021-03-30,02:01PM,Teslas first-quarter sales likely hampered by ...,https://www.marketwatch.com/story/teslas-first...,MarketWatch,0.151,0.849,0.000,-0.1531,Strong Negative
196,TSLA,2021-03-30,12:51PM,11 Most Valuable Car Companies in the World,https://finance.yahoo.com/news/11-most-valuabl...,Insider Monkey,0.000,0.674,0.326,0.5256,Strong Negative
197,TSLA,2021-03-30,11:28AM,Tesla Looking to Make 5 Semi Trucks Per Week i...,https://www.thestreet.com/investing/tesla-look...,TheStreet.com,0.000,1.000,0.000,0.0000,Strong Negative
198,TSLA,2021-03-30,11:24AM,Stock Market Today With Jim Cramer: Don't Try ...,https://www.thestreet.com/investing/stock-mark...,TheStreet.com,0.000,1.000,0.000,0.0000,Strong Positive
